In [1]:
import pyvisa

### Connect the instrument

Connect the Agilent E4980A LCR to the computer through the USB cable connected to the back of the instrument. 
If properly connected, the following cell should print a string. If the return of the print statement is empty, check that the LCr is actually turned on and make sure all cables are connected. 

In [2]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

()


### Communicate with the instrument

Select the corrent resource from the list above. If the connection is properly made, the retunr of the print statement in the following cell should be the name of the LCR **_Agilent Technologies,E4980A,MY46205278,A.02.20_**

In [3]:
my_instrument = rm.open_resource('USB0::0x0957::0x0909::MY46205278::INSTR')
print(my_instrument.query('*IDN?'))

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [4]:
my_instrument.read_termination = '\n'
my_instrument.write_termination = '\n'
my_instrument.query('*IDN?')

'Agilent Technologies,E4980A,MY46205278,A.02.20'

In [24]:
my_instrument.timeout= 100000

### Initialize the LCR Instrument

* Clear previous states and resets the instrument settings

In [6]:
my_instrument.write('*RST; *CLS')

(11, <StatusCode.success: 0>)

* Enable the display to update itself when a change is made

In [7]:
my_instrument.write(':DISP:ENAB')

(11, <StatusCode.success: 0>)

* Configure the instrument to automatically perform continuous measurements

In [8]:
my_instrument.write(':INIT:CONT')

(11, <StatusCode.success: 0>)

* Switch the instrument triggering function to 'EXT' in order to control the measurements remotely

In [9]:
my_instrument.write(':TRIG:SOUR EXT')

(15, <StatusCode.success: 0>)

### Set the experimental conditions

* Choose the function we want to use for the experimental run. In this case we are going to be running an Electrochemical Impedance Spectrum by measuring the magnitude and the phase angle - in radians

In [10]:
my_instrument.write(':FUNC:IMP:TYPE ZTR')

(19, <StatusCode.success: 0>)

* Set the input voltage amplitude by runnning the following cell. 

        Note: make sure there is a space in the string between the command and the amplitude level

In [11]:
my_instrument.write(':VOLT:LEVEL 0.01')

(17, <StatusCode.success: 0>)

* Set the measurement time to medium

In [12]:
my_instrument.write(':APER MED')

(10, <StatusCode.success: 0>)

* Set the impedance range to automatic

In [13]:
my_instrument.write(':FUNC:IMP:RANGE:AUTO ON')

(24, <StatusCode.success: 0>)

* Set the frequency you want to run the impedance measurement at

        Note:  make sure there is a space in the string between the command and the frequency

* Displays the list page. Once you set the list of freqeuncies you want to scan over, the list page should be automatically updated

In [14]:
my_instrument.write(':DISP:PAGE LIST')

(16, <StatusCode.success: 0>)

    Note: the cell below sets the data format to * ascii*.

In [15]:
my_instrument.write(':FORM:DATA ASC')

(15, <StatusCode.success: 0>)

In [16]:
my_instrument.write(':LIST:MODE SEQ')

(15, <StatusCode.success: 0>)

* Set the frequency you want to run the impedance measurement at

        Note: make sure there is a space in the string between the command and the frequency
        Note: to use a list of frequency values as enntries in the *sweep list*, we need to convert the list into a single string and remove the brakets from it. This is what the cell below does.

In [37]:
freq_list= [20,200,2000,20000,200000,2000000] # Ohms
freq_str= str(freq_list)
freq_str= freq_str.split('[')[1].split(']')[0]
freq_str

'20, 200, 2000, 20000, 200000, 2000000'

In [18]:
my_instrument.write('LIST:FREQ ', freq_str)

(47, <StatusCode.success: 0>)

* Set the source of the memory to external. 

This tells the instrument that the data will be save externally from the instrument itself.

In [19]:
my_instrument.write(':MMEM EXT')

(10, <StatusCode.success: 0>)

We are gonna use the memory buffer as I am going to independently save the data into a single file instead of having the instrument do that for me. Giving dimensions to the buffer memory ensures that only the data collected is going to be save. 

    Note:the instrument has a setting that if less than 201 poitn are collected, the remaininga are filled with defult values 

In [29]:
my_instrument.write(':MEM:DIM DBUF, ', str(len(freq_list)))

(16, <StatusCode.success: 0>)

* Initialize the memory log for the given frequency list indicated above

In [30]:
my_instrument.write(':MEM:FILL DBUF')

(15, <StatusCode.success: 0>)

* Triggers the instrument to run the measurement

In [31]:
my_instrument.write(':TRIG:IMM')

(10, <StatusCode.success: 0>)

* Let's try and save the data we just collected

In [32]:
x= my_instrument.query_ascii_values(':MEM:READ? DBUF')

For each measurement, 4 entries are generated:

    Entry 1: Impedance Magnitude
    Entry 2: Phase Angle (rad)
    Entry 3&4: 0.0
    
The following cell removes the extra zero entries and split the remaining values in the two measurements. 

In [34]:
x= [ item for item in x if item!= 0.0]
magnitude= [x[i] for i in range(len(x)) if i % 2 == 0]
phase= [x[i] for i in range(len(x)) if i % 2 != 0]

* Let's clean the memory buffer 

In [36]:
my_instrument.write(':MEM:CLE DBUF')

(14, <StatusCode.success: 0>)

* Return to measurement page

In [ ]:
my_instrument.write(':DISP:PAGE MEAS')

___
___

### _Different ways of indicating the frequency(ies) to run_

In [177]:
# 1 single frequncy in command string

## This is how you specify a single frequency

my_instrument.write(':FREQ:CW 20')

(12, <StatusCode.success: 0>)

In [111]:
# 2 Multiple frequency in command string

# separate each frequency using a comma

my_instrument.write('LIST:FREQ 20, 50,100,1E3')

(25, <StatusCode.success: 0>)

In [34]:
# 3 Iterate over frequencies by using a separate frequency variable

## In order to iterate over different frequencies, we need to change the command string
freq= 30
my_instrument.write('FREQ:CW ', str(freq))

(10, <StatusCode.success: 0>)